<a href="https://colab.research.google.com/github/amylynnn/weatherwise-Amylynn-Sophie/blob/main/workingcodes2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from datetime import datetime, timedelta

API_KEY = '7cf20335110caaf78db0fecb31852d45'

def get_weather_data(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def create_temperature_visualisation(data, city):
    temps = [entry['main']['temp'] for entry in data['list']]
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]

    plt.figure(figsize=(12,6))
    plt.plot(times, temps, marker='o', linestyle='-', color='#FF6F61', linewidth=2, label='Temperature (°C)')

    for x, y in zip(times, temps):
        plt.annotate(f"{y:.1f}°C", (x, y), textcoords="offset points", xytext=(0,8), ha='center', fontsize=8)

    plt.title(f"24-Hour Temperature Forecast for {city.title()}", fontsize=16)
    plt.xlabel("Date & Time", fontsize=12)
    plt.ylabel("Temperature (°C)", fontsize=12)
    plt.xticks(rotation=45, fontsize=9)
    plt.yticks(fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.legend()
    plt.show()

def create_precipitation_visualisation(data, city):
    rain_chances = []
    times = []

    for entry in data['list']:
        rain = entry.get('rain', {}).get('3h', 0)
        rain_chances.append(rain)
        times.append(datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S"))

    plt.figure(figsize=(12,6))
    bars = plt.bar(times, rain_chances, width=0.1, color='#69B3E7', edgecolor='black')

    for bar, value in zip(bars, rain_chances):
        if value > 0:
            plt.text(bar.get_x() + bar.get_width()/2, value + 0.1, f"{value:.1f}mm",
                     ha='center', fontsize=8)

    plt.title(f"Precipitation Forecast for {city.title()} (Next 5 Days)", fontsize=16)
    plt.xlabel("Date & Time", fontsize=12)
    plt.ylabel("Rain (mm in 3h)", fontsize=12)
    plt.xticks(rotation=45, fontsize=9)
    plt.yticks(fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

def parse_weather_question(question):
    match = re.search(r"(rain|snow|clear|cloudy).* in (\w+(?: \w+)*) (today|tomorrow)?", question.lower())
    if match:
        condition = match.group(1)
        city = match.group(2)
        day = match.group(3) or 'today'
        return {'condition': condition, 'city': city, 'day': day}
    return None

def generate_weather_response(parsed):
    if not parsed:
        return "⚠️ Sorry, I couldn't understand your question. Try asking like: 'Will it rain in Paris tomorrow?'"

    condition = parsed['condition']
    city = parsed['city']
    day = parsed['day']

    try:
        data = get_weather_data(city)
        target_date = datetime.utcnow().date()
        if day == 'tomorrow':
            target_date += timedelta(days=1)

        found = False
        for entry in data['list']:
            entry_date = datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S").date()
            weather_desc = entry['weather'][0]['description'].lower()
            if entry_date == target_date and condition in weather_desc:
                found = True
                break

        if found:
            return f"✅ Yes, it looks like it will be {condition} in {city.title()} {day}."
        else:
            return f"❌ No, it doesn't look like it will be {condition} in {city.title()} {day}."

    except Exception as e:
        return f"❌ Could not retrieve weather for {city.title()}: {e}"

def interactive_ui():
    city_input = widgets.Text(value='', placeholder='Enter city (e.g. Paris)', description='City:')
    question_input = widgets.Text(value='', placeholder='e.g. Will it rain in Paris tomorrow?', description='Ask:')
    output_area = widgets.Output()

    forecast_btn = widgets.Button(description="📊 Show Forecast Charts", button_style='info')
    question_btn = widgets.Button(description="🧠 Answer My Question", button_style='success')

    def on_forecast_button_click(_):
        with output_area:
            clear_output()
            try:
                data = get_weather_data(city_input.value)
                create_temperature_visualisation(data, city_input.value)
                create_precipitation_visualisation(data, city_input.value)
            except Exception as e:
                print(f"❌ Error: {e}")

    def on_question_button_click(_):
        with output_area:
            clear_output()
            parsed = parse_weather_question(question_input.value)
            response = generate_weather_response(parsed)
            print(response)

    forecast_btn.on_click(on_forecast_button_click)
    question_btn.on_click(on_question_button_click)

    question_section = widgets.VBox([question_input, question_btn], layout=widgets.Layout(margin='10px 0 10px 0'))
    forecast_section = widgets.VBox([city_input, forecast_btn], layout=widgets.Layout(margin='10px 0 10px 0'))

    main_box = widgets.VBox([
        widgets.HTML("<h2 style='text-align:center; color:#2E8B57;'>🌤️ Interactive Weather Assistant</h2>"),
        question_section,
        widgets.HTML("<hr style='border: 1px solid #ccc;'>"),
        forecast_section,
        output_area
    ], layout=widgets.Layout(
        border='2px solid #2E8B57',
        padding='15px',
        width='650px',
        margin='auto',
        box_shadow='2px 2px 5px rgba(0,0,0,0.2)'
    ))

    display(main_box)

interactive_ui()
